In [1]:
import pinecone
from tqdm.autonotebook import tqdm
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Pinecone

/Users/skarlekar/Documents/theWorks/digital-sme/.venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY=os.environ['PINECONE_API_KEY']
PINECONE_ENV=os.environ['PINECONE_ENV']

In [3]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

In [19]:
index_name = 'uw-guide-index'

In [20]:

if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=1536,
        metric='cosine'
    )
    print("Underwriting Guide Index created!")
else:
  print("Underwriting Guide Index aleady exists!")

In [21]:
loader = DirectoryLoader("./docs/", glob="**/*.pdf")
documents = loader.load()


In [22]:
print (len(documents))

38


In [25]:
text_splitter = CharacterTextSplitter(
    separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len
)
texts = text_splitter.split_documents(documents)

Created a chunk of size 1838, which is longer than the specified 1000
Created a chunk of size 1195, which is longer than the specified 1000
Created a chunk of size 1094, which is longer than the specified 1000
Created a chunk of size 1204, which is longer than the specified 1000
Created a chunk of size 1579, which is longer than the specified 1000
Created a chunk of size 1284, which is longer than the specified 1000
Created a chunk of size 1083, which is longer than the specified 1000
Created a chunk of size 1296, which is longer than the specified 1000
Created a chunk of size 1905, which is longer than the specified 1000
Created a chunk of size 2596, which is longer than the specified 1000
Created a chunk of size 1153, which is longer than the specified 1000
Created a chunk of size 1209, which is longer than the specified 1000
Created a chunk of size 1288, which is longer than the specified 1000
Created a chunk of size 1355, which is longer than the specified 1000
Created a chunk of s

In [26]:
print(len(texts))

999


In [27]:
embeddings = OpenAIEmbeddings()

In [28]:
knowledge_base = Pinecone.from_documents(documents=texts,
                                     embedding=embeddings,
                                    index_name=index_name)

In [29]:
user_question = (
    "What is ACheck, when is it used, what are the requirements for using ACheck?"
)
resultset = knowledge_base.similarity_search(user_question)
print(len(resultset))
for x in resultset:
    print(x)

4
page_content='When do you use the ACheck application?\nAs soon as you receive an application request and the necessary tax identification numbers and/or social security numbers, enter the information into ACheck.\nIf you have not yet identified all parties at this stage, you must enter all Key Principals, Principals, and Guarantors of the Borrower into ACheck as soon as you identify them.\nIf more than 90 days pass between the initial ACheck and Commitment Date, you must repeat an ACheck.\nHow do you view ACheck results?\nThe ACheck application will provide either a “Continue Processing” or “Do Not Process” electronic response instantaneously.\n“Continue Processing” Response\nYou must receive a “Continue Processing” response in order to proceed with the application.\nThis does not mean that the Borrower, Key Principal, Principal, Guarantor, or Principal is approved; you are still required to complete full Mortgage Loan credit underwriting.\n“Do Not Process” Response' metadata={'sourc

In [30]:
llm = OpenAI(temperature=0)
chain = load_qa_chain(llm, chain_type="stuff")
response = chain.run(input_documents=resultset, question=user_question)
print(response)

 ACheck is a lender due diligence application performed for the Borrower, Key Principal, and Principal. It is used as soon as you receive an application request and the necessary tax identification numbers and/or social security numbers. The requirements for using ACheck are that you must receive a "Continue Processing" response for the Borrower, each Key Principal of the Borrower, each Principal of the Borrower, each Guarantor, and any person who owns or controls an entity Key Principal.
